# Data Processing for Crowd Annotation Pipeline

1. Download job report from Figure Eight
2. Download annotated images from report
3. Clean up annotations (remove small objects, fill small holes, sequentially label the annotations)
4. Process annotations and raw images to match each other in size. Can either:
    - chop raw images into corresponding pieces
    - recombine annotation subimages into full-size images
5. Combine raw and annotated images into npz format
6. Images are now ready to be used as training data!

Files are named by these scripts such that the code blocks can run back-to-back with minimal input. For this reason, it is recommended that users run through the whole pipeline before processing another set of images. The user can specify a few directory names and the "identifier" used in pre-annotation and run all cells in the notebook; alternate folder names can be used but this is not recommended.

To function properly, your working folder should contain subfolders:
- json_logs  
    - log from overlapping_chopper ({identifier}\_overlapping\_chopper_log.json)
- raw images (can be named "raw" or something else)

The user will also need to supply:
- job ID for the data to download from figure eight
- API key for figure eight
- "identifier" to access correct json logs and name files correctly

If the default folder names are used, by the end of this pipeline, the working folder (base_dir) will contain the following new files and directories:

- CSV/job report downloaded from figure eight
- annotations  
    - downloaded montages from figure eight, cleaned
- (optional) chopped raw images
- .npz file containing training data


In [13]:
#import statements
import os

from deepcell_toolbox.post_annotation.download_csv import download_and_unzip, save_annotations_from_csv
from deepcell_toolbox.post_annotation.clean_montages import clean_montages, relabel_montages, convert_grayscale_all
from deepcell_toolbox.post_annotation.montages_to_movies import raw_movie_maker
#from deepcell_toolbox.post_annotation.overlapping_stitcher import overlapping_stitcher

from deepcell_toolbox.utils.data_utils import make_training_data

In [2]:
#set working directory
#base_dir = "/base/directory/path/here"
#raw_dir = "/base/directory/path/here/folder_with_fullsize_raw_images"

base_dir = "/gnv_home/data/2D_upload_test/set0"
raw_dir = "/gnv_home/data/2D_upload_test/set0/raw"

#identifier given during pre-annotation pipeline; if you're not sure, it's also in the job report csv
identifier = "ecoli_s0"

## 1. Download job report from Figure Eight
By default, this script will download, unzip, and rename the full report from Figure Eight as a .csv file. However, the user can change the report type if one of the other report options is more suitable for their use. (support for other report types not guaranteed with version 0 of this notebook)

The user can specify where the zip file should be downloaded and the .csv extracted; by default, the .csv file will be put into a subfolder named CSV (likely the same folder that contained the input data; the CSV files are named to prevent confusion). The report CSV will be renamed "job_{job_number}\_{type of report}\_report.csv".

#### From Figure Eight website:
full - Returns the Full report containing every judgment

aggregated - Returns the Aggregated report containing the aggregated response for each row

json - Returns the JSON report containing the aggregated response, as well as the individual judgments

gold_report - Returns the Test Question report

workset - Returns the Contributor report

source - Returns a CSV of the source data uploaded to the job

In [3]:
job_id_to_download = 1366009
job_type = "full"

In [ ]:
download_and_unzip(job_id_to_download, base_dir)

## 2. Use report to download annotations
This script uses the information in the report to download each annotation. Montage annotations will be saved in the "annotations" subfolder (it will be created for you by the script).

Raw images that could not be annotated (those with "broken_link = True") will not be downloaded in this step. If a job contains rows with broken links, the information will be put into two csv files: one, "job_number_full_report_broken_links.csv", contains all of the metadata from the full job report, in case the user wants to inspect this for a pattern in the broken links. The other, "job_number_reupload.csv", has only the information used to upload the images originally (identifier and annotation_url). If the images are suitable for annotation, the user can easily add this csv to the figure eight job and obtain annotations for those images. (Alternatively, the user may need to go through part of the pre-annotation pipeline to adequately fix and reupload the images in question.)

If there are no broken links in the job, or if images with broken links instead of annotations have annotations later on in the job report (if the user has reuploaded those rows to the job), the secondary csv creation will not be triggered.

This function returns a list of the image names of any images with broken links; this list will be used later in the pipeline to automatically skip images when stitching images together or making training data.

In [4]:
csv_dir = os.path.join(base_dir, "CSV")
csv_path = os.path.join(csv_dir, "job_" + str(job_id_to_download) + "_" + job_type + "_report.csv")

#csv_path = "/example/path/CSV/job_number_full_report.csv"

annotation_save = os.path.join(base_dir, "annotations")

In [ ]:
images_to_drop = save_annotations_from_csv(csv_path, annotation_save)

## 3. Clean up annotations
First, the RGB annotation is converted into grayscale, simplifying downstream use of the annotation.

Next, small changes to the morphology of the image are made. Sometimes during annotation, small holes or stray annotations will be submitted, as artifacts of the annotation process. However, these holes or stray pixels don't correspond to what should be annotated, so in this step, we use sci-kit image to fix these small mistakes.

Currently uses the old "clean_montage" function; this may change in future versions of notebook.

After cleaning the annotation, user can optionally run "relabel_montages" block, which will relabel the annotations sequentially (eg, perhaps the annotator decided to use the labels 3, 5, and 7 to label cells; this code block would remake the image with labels 1, 2, and 3).

The cleaned and relabled annotations will overwrite the downloaded annotations.

In [5]:
annotations_folder = annotation_save
#annotations_folder = "/base/directory/path/here/wherever_you_moved_the_annotations"

In [6]:
convert_grayscale_all(annotations_folder)

In [7]:
clean_montages(annotations_folder)

In [8]:
#optional
relabel_montages(annotations_folder)

## 4. Make raw and annotated images same size
Depending on which option you choose (for 2D, option 2 is recommended), the file organization may change
In each option:
 - code block 1: read/set parameters
 - code block 2: modify images
    - chopper should chop all
    - move anything not in images_to_drop into training folder
    - stitcher should skip stitching anything that contains an image_to_drop
    - stitcher should move files into training folder?

### Option 1: Chop raw images into pieces to match annotation size

In [ ]:
# move overlapping_chopper base function into utils
# read json parameters

In [ ]:
# use the chop parameters to populate training_direcs

### Option 2: Recombine annotations to match original raw image size
- basic functionality to recombine annotations
- tries to combine labels for annotations that overlap to reduce cells that are annotated half as one thing and half as another
- does not support images_to_drop yet
- may encounter problems if original image size is not divisible by num_segments
    - this would be a problem for npz formation; image stitching would work fine, but the image sizes may be off
- overlapping_stitcher_2D wrapper definition will live here for now until more features are added, then it will be moved into overlapping_stitcher.py    

In [ ]:
import os
import stat
import sys

def overlapping_stitcher_2D(pieces_dir, num_images, save_dir, num_x, num_y, overlap_perc, identifier):
    '''
    wrapper function for using the overlapping stitcher on 2D annotations from figure eight
    '''
        
    #directories
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
        #add folder modification permissions to deal with files from file explorer
        mode = stat.S_IRWXO | stat.S_IRWXU | stat.S_IRWXG
        os.chmod(save_dir, mode)
    
    for image in range(num_images):
        sub_img_format = "_img_{0:02d}".format(image)

        save_name = identifier + sub_img_format + "_annotation.png"       
        
        sub_img_format = "{0}" + sub_img_format + "_x_{1:02d}_y_{2:02d}_annotation.png" 
        
        overlapping_stitcher(pieces_dir, save_dir, num_x, num_y, overlap_perc, identifier, sub_img_format, save_name)

## 5. Combine raw and annotated images into npz format
- need to figure how to structure training_direcs
- not really supported yet in this (2D) notebook, this is more of a placeholder code block


In [21]:
import os
import stat
import sys
import pathlib

#get info from json logs
#log_folder = os.path.join(base_dir, "json_logs")
#chop_params = read_json_params_montage(log_folder, identifier)
#number of frames per montage
#num_frames = montage_params[0]
#segments the original image was chopped into:
#num_x = montage_params[1]
#num_y = montage_params[2]

# Load data
base_dir = '/gnv_home/data/overlapping_stitcher_test'
output_directory = os.path.join(base_dir, "npz")
#output_directory = '/gnv_home/data/Ed/3T3/set0'
file_name_save = os.path.join(output_directory, 'ecoli_s0_retry_same.npz')

#Training directories are organized according to location within an image
#if there are any movies that shouldn't be included in the npz
#(unsuitable for training, or don't need to be tracked), put them in "samples_to_drop"
samples_to_drop = []
training_direcs = ["training"]
#training_direcs = ['x_0{}_y_0{}'.format(i,j) for i in range(num_x) for j in range(num_y)]
#training_direcs = [x for x in training_direcs if x not in samples_to_drop]
channel_names = [""] # Commonality in raw filenames

# Create output directory, if necessary
pathlib.Path(output_directory).mkdir(parents=True, exist_ok=True)
#mode = stat.S_IRWXO | stat.S_IRWXU | stat.S_IRWXG
#os.chmod(output_directory, mode)

In [22]:
# Create the training data
make_training_data(
    direc_name = base_dir,
    file_name_save = file_name_save,
    channel_names = channel_names,
    dimensionality = 2,
    training_direcs = training_direcs,
    raw_image_direc = "raw",
    annotation_direc = "stitched_annotations",
    annotation_name = "",
    border_mode = "same",
    output_mode = "conv",
    reshape_size = None,
    verbose = True)

Number of features: 3
Number of training data points: 2585664
Class weights: [1.]
